In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import string
import re

from tqdm import tqdm

In [11]:
import json
from pandas.io.json import json_normalize

def mapping_label(x):
      return lab['label'][x]

In [12]:
def get_labels_df(my_json, lbls):
    data = []
    for line in open(my_json, 'r',errors='ignore'):
        data.append(json.loads(line))
    usr_list=[]
    for i in range(len(data)):
        for j in data[i]['annotations']:
            usr_list.append(j['user'])
    
    usr_dict = {}
    for i in set(usr_list):
        usr_dict[i]=[]
    
    for i in range(len(data)):
        for k,v in usr_dict.items():
            usr_dict[k] = []
            for j in range(len(data[i]['annotations'])):
                if data[i]['annotations'][j]['user']==k:
                    usr_dict[k].append(data[i]['annotations'][j]['label'])
                data[i][k]=usr_dict[k]
    labels=lbls
    lab = pd.read_csv(labels)
    lab = lab.to_dict()
    lab_dict = dict(zip([lab['number'][j] for j in lab['number']], [lab['label'][j] for j in lab['label']]))
    for k,v in usr_dict.items():
        for i in range(len(data)):
            try:
                data[i][k]= [ lab_dict.get(item,item) for item in data[i][k]]
            except:
                pass
    df = pd.DataFrame.from_dict(data)
    unique_users = list(set(usr_list))
    df['lab_final'] = [[]] * len(df)
    for user in unique_users:
        for i in range(len(df)):
            if df[user][i]:
                df['lab_final'][i] = df[user][i]  
    df= df[df['lab_final'].str.len() != 0]
    df= df[df['lab_final']==df['lab_final']] 
    #-------------------------------------------
    print(list(usr_dict.keys()))
    df.drop(list(usr_dict.keys()), axis = 1, inplace=True)
    #-------------------------------------------  
    print(df.shape)
    return df

In [13]:
main_df = pd.read_csv('en_interim_01_05_DV_IN_with_summary_wrapping_final.csv', keep_default_na=False)
main_df.head(2)

,Unnamed: 0,Unnamed: 0.1,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,...,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text,summary,summary_wrap
0,0,0,25090,896079819,19200101,192001,1920,1920.0027,ELI,RETIRED,...,70178.0,18.5333,73.8667,-2108361,20200101044500,https://punemirror.indiatimes.com/pune/cover-s...,pmc threatens to seize house of ex i t comm wh...,PMC threatens to seize house of ex I-T comm wh...,"In this light, he had also sent a legal notice...","In this light, he had also sent a legal notice..."
1,1,1,33280,896089642,19200101,192001,1920,1920.0027,COP,POLICEMAN,...,17924.0,27.2494,79.3139,10451923,20200101070000,https://www.mid-day.com/articles/mentalhood-ta...,mentalhood tandav the family man 2 breathe 2 s...,"\nMentalhood, Tandav, The Family Man 2, Breath...","Mirzapur 2Cast: Ali Fazal, Shweta Tripathi, Ra...","Mirzapur 2Cast: Ali Fazal, Shweta Tripathi, Ra..."


In [14]:
my_json="hoa.json"
lbls="labels_news.csv"

In [15]:
df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

[8]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,summary_wrap,annotations,meta,annotation_approver,lab_final
0,3576,The court observed that the man wanted to star...,"[{'label': 21, 'user': 8}]",{},None,[DV]
1,3637,The children were rescued as part of 'Operatio...,"[{'label': 27, 'user': 8}]",{},None,[Irrelevant]
2,3556,summary_wrap,"[{'label': 27, 'user': 8}]",{},None,[Irrelevant]
3,3558,"Mirzapur 2Cast: Ali Fazal, Shweta Tripathi, Ra...","[{'label': 27, 'user': 8}]",{},None,[Irrelevant]
4,3559,Nagpur: Amid cloudy day and chilling night the...,"[{'label': 27, 'user': 8}]",{},None,[Irrelevant]


In [16]:
data1 = pd.merge(main_df, df, on="summary_wrap", how='inner')

In [17]:
print(data1)

     Unnamed: 0  Unnamed: 0.1  index  GLOBALEVENTID   SQLDATE  MonthYear  \
0             0             0  25090      896079819  19200101     192001   
1             1             1  33280      896089642  19200101     192001   
2             2             2  34788      896095544  19200101     192001   
3             3             3  35500      896092971  19200101     192001   
4             4             4  47722      896113974  19200101     192001   
..          ...           ...    ...            ...       ...        ...   
519         528           528  57900      900694120  20200122     202001   
520         529           529  58651      900692176  20200122     202001   
521         530           530  61449      900699877  20200122     202001   
522         531           531  65063      900697954  20200122     202001   
523         532           532  69906      900710254  20200122     202001   

     Year  FractionDate Actor1Code      Actor1Name  ...  \
0    1920     1920.0027     

In [18]:
data1.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

In [19]:
print(data1)

     Unnamed: 0.1  index  GLOBALEVENTID   SQLDATE  MonthYear  Year  \
0               0  25090      896079819  19200101     192001  1920   
1               1  33280      896089642  19200101     192001  1920   
2               2  34788      896095544  19200101     192001  1920   
3               3  35500      896092971  19200101     192001  1920   
4               4  47722      896113974  19200101     192001  1920   
..            ...    ...            ...       ...        ...   ...   
519           528  57900      900694120  20200122     202001  2020   
520           529  58651      900692176  20200122     202001  2020   
521           530  61449      900699877  20200122     202001  2020   
522           531  65063      900697954  20200122     202001  2020   
523           532  69906      900710254  20200122     202001  2020   

     FractionDate Actor1Code      Actor1Name Actor1CountryCode  ...  \
0       1920.0027        ELI         RETIRED                    ...   
1       1920.0027

In [20]:
my_json="meghana.json"
lbls="labels_news.csv"

In [21]:
df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

[7]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,summary_wrap,annotations,meta,annotation_approver,lab_final
0,4065,"puneUpdated: Jan 23, 2020 16:05 ISTPost arrest...","[{'label': 34, 'user': 7}]",{},None,[Irrelevant]
1,4057,summary_wrap,"[{'label': 34, 'user': 7}]",{},None,[Irrelevant]
2,4058,"Discussing the report with media persons, Jana...","[{'label': 34, 'user': 7}]",{},None,[Irrelevant]
3,4059,The investigating officers are also suspecting...,"[{'label': 30, 'user': 7}]",{},None,[Murder/Suicide]
4,4060,"After alleging sexual harassment by Gogoi, the...","[{'label': 34, 'user': 7}]",{},None,[Irrelevant]


In [22]:
data2 = pd.merge(main_df, df, on="summary_wrap", how='inner')

In [23]:
print(data2)

     Unnamed: 0  Unnamed: 0.1   index  GLOBALEVENTID   SQLDATE  MonthYear  \
0           533           533   82894      900732117  20200122     202001   
1           534           534   87054      900741136  20200122     202001   
2           535           535   98295      900757401  20200122     202001   
3           536           536   98653      900757759  20200122     202001   
4           537           537  106315      900771544  20200122     202001   
..          ...           ...     ...            ...       ...        ...   
525        1052          1052   49221      904624427  20200208     202002   
526        1053          1053   51430      904627931  20200208     202002   
527        1054          1054   54268      904631524  20200208     202002   
528        1055          1055   57441      904637771  20200208     202002   
529        1056          1056   59242      904640782  20200208     202002   

     Year  FractionDate Actor1Code        Actor1Name  ...  \
0    2020     

In [24]:
data2.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

In [25]:
my_json="nikhel.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data3 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data3.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

[4]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
my_json="seyi.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data4 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data4.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

[3]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [27]:
my_json="shreyasi.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data5 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data5.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

[5]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
my_json="shrusti.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data8 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data8.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

[9, 10]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
my_json="manasvi.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data9 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data9.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

[10]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
my_json="saloni.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data10 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data10.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True) 

[11]
(467, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
data1.to_csv("hoa.csv")

In [32]:
data2.to_csv("meghana.csv")

In [33]:
data3.to_csv("nikhel.csv")

In [34]:
data4.to_csv("seyi.csv")

In [35]:
data5.to_csv("shreyasi.csv")

In [36]:
data8.to_csv("shrusti.csv")

In [37]:
data9.to_csv("manasvi.csv")

In [38]:
data10.to_csv("saloni.csv")

In [43]:
all_filenames = ['hoa.csv','meghana.csv','nikhel.csv','seyi.csv', 'shreyasi.csv','shubhang.csv','harini.csv','shrusti.csv','manasvi.csv', 'saloni.csv']

In [44]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
del combined_csv["annotations"]
del combined_csv["annotation_approver"]
combined_csv.to_csv( "english_5K_label_final.csv", index=False, encoding='utf-8-sig')

In [45]:
harinis = pd.read_csv("english_5K_label_final.csv")

In [48]:
print(harinis)

      Unnamed: 0  Unnamed: 0.1  index  GLOBALEVENTID   SQLDATE  MonthYear  \
0              0             0  25090      896079819  19200101     192001   
1              1             1  33280      896089642  19200101     192001   
2              2             2  34788      896095544  19200101     192001   
3              3             3  35500      896092971  19200101     192001   
4              4             4  47722      896113974  19200101     192001   
...          ...           ...    ...            ...       ...        ...   
5222         490          5181  15930      925260114  20200524     202005   
5223         491          5182  15933      925260117  20200524     202005   
5224         492          5183  16500      925260684  20200524     202005   
5225         493          5184  17546      925262321  20200524     202005   
5226         494          5185  17696      925262471  20200524     202005   

      Year  FractionDate Actor1Code Actor1Name  ... ActionGeo_Lat  \
0     

In [39]:
my_json="shubhang.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data6 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data6.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True)

[6]
(501, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [40]:
my_json="harini.json"
lbls="labels_news.csv"

df = get_labels_df(my_json,lbls)
df=df.rename(columns={'text':'summary_wrap'})
df.head()

data7 = pd.merge(main_df, df, on="summary_wrap", how='inner')
data7.drop(['Unnamed: 0','id','meta'], axis = 1, inplace=True)

[1]
(500, 6)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
data6.to_csv("shubhang.csv")

In [42]:
data7.to_csv("harini.csv")

In [ ]:
all_filenames = ['hoa.csv','meghana.csv','nikhel.csv','seyi.csv', 'shreyasi.csv','shubhang.csv','harini.csv','shrusti.csv','manasvi.csv', 'saloni.csv']
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
del combined_csv["annotations"]
del combined_csv["annotation_approver"]
combined_csv.to_csv( "english_5K_label_final.csv", index=False, encoding='utf-8-sig')